In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/tianyifan/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-p

In [5]:
CLIENT_ID = 'ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I' # your Foursquare ID
CLIENT_SECRET = 'R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I
CLIENT_SECRET:R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ


In [44]:
#search for venues around WeWork location at 1701 Rhode Island Ave NW

LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.906945, 
    -77.038768, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.906945,-77.038768&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0e4d84c8cff2001b0f32ad'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Connecticut Avenue - K Street',
  'headerFullLocation': 'Connecticut Avenue - K Street, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 115,
  'suggestedBounds': {'ne': {'lat': 38.9114450045, 'lng': -77.03299597120697},
   'sw': {'lat': 38.9024449955, 'lng': -77.04454002879304}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6a7385f964a5207dcd1fe3',
       'name': 'Iron Gate',
       'location': {'address': '1734 N St NW',
        'lat': 38.906952576466864,
        'lng': -77.04001893727637,
      

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Iron Gate,Mediterranean Restaurant,38.906953,-77.040019
1,Shobha DC,Health & Beauty Service,38.905519,-77.040096
2,The Jefferson Hotel,Hotel,38.905932,-77.036856
3,Tabard Inn,Hotel,38.907364,-77.040013
4,&pizza,Pizza Place,38.906407,-77.041065


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [13]:
nearby_venues.head(100)

,name,categories,lat,lng
0,Iron Gate,Mediterranean Restaurant,38.906953,-77.040019
1,Shobha DC,Health & Beauty Service,38.905519,-77.040096
2,The Jefferson Hotel,Hotel,38.905932,-77.036856
3,Tabard Inn,Hotel,38.907364,-77.040013
4,&pizza,Pizza Place,38.906407,-77.041065
5,National Geographic Society,Museum,38.905190,-77.037259
6,The Coffee Bar,Coffee Shop,38.905721,-77.038662
7,The Mayflower Hotel Concierge Lounge,Hotel Bar,38.904738,-77.039225
8,CAVA,Mediterranean Restaurant,38.906639,-77.042132
9,Quill,Hotel Bar,38.906073,-77.036857


In [50]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
Hotel,7,7,7
Coffee Shop,5,5,5
Mediterranean Restaurant,4,4,4
Hotel Bar,4,4,4
Sandwich Place,4,4,4
Italian Restaurant,3,3,3
Lounge,3,3,3
American Restaurant,3,3,3
Café,3,3,3


In [51]:
#search for venues around WeWork location at 901 North Glebe Road, Arlington, VA
LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.882077, 
    -77.115196,  
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.882077,-77.115196&radius=500&limit=100'

In [52]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Total Wine & More,Wine Shop,38.879881,-77.115051
1,Nando's Peri-Peri,Portuguese Restaurant,38.880175,-77.113520
2,Sweet Leaf,Salad Place,38.880349,-77.114746
3,Big Buns,Burger Joint,38.880764,-77.113693
4,Uncle Julio's Rio Grande Cafe,Mexican Restaurant,38.882473,-77.112973


In [53]:
nearby_venues.head(100)

,name,categories,lat,lng
0,Total Wine & More,Wine Shop,38.879881,-77.115051
1,Nando's Peri-Peri,Portuguese Restaurant,38.880175,-77.113520
2,Sweet Leaf,Salad Place,38.880349,-77.114746
3,Big Buns,Burger Joint,38.880764,-77.113693
4,Uncle Julio's Rio Grande Cafe,Mexican Restaurant,38.882473,-77.112973
5,Republik Coffee Bar,Coffee Shop,38.880628,-77.113412
6,Zoës Kitchen,Mediterranean Restaurant,38.882187,-77.112272
7,Crafthouse,Bar,38.881518,-77.115628
8,Mussel Bar & Grille,American Restaurant,38.880091,-77.114595
9,The Melting Pot,Fondue Restaurant,38.884020,-77.117210


In [54]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
Coffee Shop,6,6,6
American Restaurant,3,3,3
Food Truck,3,3,3
Bank,3,3,3
Café,3,3,3
Hotel,3,3,3
Shipping Store,2,2,2
Sandwich Place,2,2,2
Mexican Restaurant,2,2,2


In [96]:
#search for venues around WeWork location at 700 K street NW Washington, DC 20003


LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.901914, 
    -77.022196, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.901914,-77.022196&radius=500&limit=100'

In [97]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Apple Carnegie Library,Electronics Store,38.902532,-77.022997
1,The Capital Burger,Burger Joint,38.903107,-77.021751
2,RPM Italian,Italian Restaurant,38.902220,-77.020940
3,La Colombe Coffee Roasters,Coffee Shop,38.901054,-77.020102
4,Taco Bamba,Taco Place,38.901064,-77.022682


In [98]:
nearby_venues.head(100)

,name,categories,lat,lng
0,Apple Carnegie Library,Electronics Store,38.902532,-77.022997
1,The Capital Burger,Burger Joint,38.903107,-77.021751
2,RPM Italian,Italian Restaurant,38.902220,-77.020940
3,La Colombe Coffee Roasters,Coffee Shop,38.901054,-77.020102
4,Taco Bamba,Taco Place,38.901064,-77.022682
5,Shouk,Vegetarian / Vegan Restaurant,38.902737,-77.020600
6,Kinship,New American Restaurant,38.903305,-77.021797
7,SoulCycle Mount Vernon,Cycle Studio,38.901930,-77.020469
8,CAVA,Mediterranean Restaurant,38.899971,-77.022314
9,Sixth & I Historic Synagogue,Synagogue,38.900884,-77.020171


In [99]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
American Restaurant,8,8,8
Italian Restaurant,5,5,5
Hotel,5,5,5
Ramen Restaurant,4,4,4
Bar,3,3,3
Sandwich Place,3,3,3
Pizza Place,3,3,3
Mediterranean Restaurant,3,3,3
Coffee Shop,3,3,3


In [102]:
#search for venues around Regus location at 1655 NORTH FORT MYER DRIVE, SUITE 700, ARLINGTON, VIRGINIA, 22209


LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.893956, 
    -77.071495,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.893956,-77.071495&radius=500&limit=100'

In [103]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Compass Coffee,Coffee Shop,38.895280,-77.071296
1,District Taco,Taco Place,38.894471,-77.074424
2,sweetgreen,Salad Place,38.896320,-77.071085
3,Freedom Park,Park,38.893851,-77.070790
4,Potbelly Sandwich Shop,Sandwich Place,38.895574,-77.070623


In [104]:
nearby_venues.head(100)

,name,categories,lat,lng
0,Compass Coffee,Coffee Shop,38.895280,-77.071296
1,District Taco,Taco Place,38.894471,-77.074424
2,sweetgreen,Salad Place,38.896320,-77.071085
3,Freedom Park,Park,38.893851,-77.070790
4,Potbelly Sandwich Shop,Sandwich Place,38.895574,-77.070623
5,CAVA,Mediterranean Restaurant,38.895503,-77.071096
6,Hyatt Centric Arlington,Hotel,38.894908,-77.072859
7,Nando's Peri-Peri,Portuguese Restaurant,38.896782,-77.071340
8,The Little Beet,Vegetarian / Vegan Restaurant,38.896571,-77.071059
9,Capriotti's Sandwich Shop,Sandwich Place,38.894565,-77.074650


In [105]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
Sandwich Place,4,4,4
Coffee Shop,4,4,4
Gym / Fitness Center,3,3,3
Hotel,3,3,3
Mediterranean Restaurant,2,2,2
Taco Place,2,2,2
Salad Place,2,2,2
New American Restaurant,1,1,1
Portuguese Restaurant,1,1,1


In [90]:
#search for venues around Regus location at 1441 L ST NW,DC

LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.904181, 
    -77.033647,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.904181,-77.033647&radius=500&limit=100'

In [91]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Madison Washington DC Hotel,Hotel,38.905354,-77.034319
1,Arepa Zone,Arepa Restaurant,38.904465,-77.031720
2,Blue Bottle Coffee,Coffee Shop,38.904317,-77.034948
3,&pizza,Pizza Place,38.902155,-77.032257
4,The Oriental Oasis,Spa,38.903363,-77.033349


In [92]:
nearby_venues.head(100)

,name,categories,lat,lng
0,The Madison Washington DC Hotel,Hotel,38.905354,-77.034319
1,Arepa Zone,Arepa Restaurant,38.904465,-77.031720
2,Blue Bottle Coffee,Coffee Shop,38.904317,-77.034948
3,&pizza,Pizza Place,38.902155,-77.032257
4,The Oriental Oasis,Spa,38.903363,-77.033349
5,Inari Salon,Salon / Barbershop,38.902586,-77.033278
6,JOE & THE JUICE,Juice Bar,38.902197,-77.034886
7,Lincoln,New American Restaurant,38.904244,-77.033064
8,Elizabeth's Gone Raw,Vegetarian / Vegan Restaurant,38.903791,-77.031363
9,Pret A Manger,Sandwich Place,38.902379,-77.033801


In [93]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
Hotel,15,15,15
Coffee Shop,7,7,7
Hotel Bar,6,6,6
Salon / Barbershop,5,5,5
Food Truck,5,5,5
Sandwich Place,4,4,4
American Restaurant,4,4,4
Sushi Restaurant,3,3,3
Vegetarian / Vegan Restaurant,2,2,2


In [94]:
#search for venues around Regus location at 1000 POTOMAC STREET NW, DC

LIMIT = 100
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.903902, 
    -77.065275, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODMN0T1XY1EQRLN5UVNQWV1BVD15AOF1BEQKWNQOALQUJO4I&client_secret=R0WSUOPR00NLUZJNY3NS50GHJQ1ROGQYKZPDNIBPKU25EIWQ&v=20180605&ll=38.903902,-77.065275&radius=500&limit=100'

In [72]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Blue Bottle Coffee,Coffee Shop,38.904379,-77.065437
1,Chaia Tacos,Taco Place,38.904029,-77.063053
2,lululemon,Clothing Store,38.905354,-77.064879
3,South Block,Juice Bar,38.903846,-77.063664
4,Luke's Lobster,Seafood Restaurant,38.905494,-77.065061


In [73]:
nearby_venues.head(100)

,name,categories,lat,lng
0,Blue Bottle Coffee,Coffee Shop,38.904379,-77.065437
1,Chaia Tacos,Taco Place,38.904029,-77.063053
2,lululemon,Clothing Store,38.905354,-77.064879
3,South Block,Juice Bar,38.903846,-77.063664
4,Luke's Lobster,Seafood Restaurant,38.905494,-77.065061
5,Falafel Inc,Falafel Restaurant,38.905476,-77.065322
6,Pizzeria Paradiso,Pizza Place,38.905028,-77.065560
7,Anthropologie,Women's Store,38.905050,-77.063720
8,Warby Parker,Optical Shop,38.905349,-77.063729
9,"Down Dog Power Yoga, LLC.",Yoga Studio,38.904716,-77.065370


In [74]:
nearby_venues.groupby('categories').count().sort_values(by=['name'],ascending=False)

,name,lat,lng
categories,,,
Coffee Shop,6,6,6
American Restaurant,5,5,5
Italian Restaurant,5,5,5
Clothing Store,4,4,4
Cosmetics Shop,3,3,3
Bakery,3,3,3
Women's Store,3,3,3
Dessert Shop,3,3,3
Tea Room,2,2,2
